In [1]:
import org.apache.spark.sql.types.{StructType, StringType}
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.streaming._


In [2]:
val inputStreamPath = "/home/jovyan/data/events-stream"
val modelPath = "/home/jovyan/models/spark-ml-model"

val dataSchema = new StructType()
    .add("tweet", StringType)

val inputDF = spark
    .readStream
    .schema(dataSchema)
    .option("maxFilesPerTrigger", 1)
    .json(inputStreamPath)

inputStreamPath = /home/jovyan/data/events-stream
modelPath = /home/jovyan/models/spark-ml-model
dataSchema = StructType(StructField(tweet,StringType,true))
inputDF = [tweet: string]


[tweet: string]

In [11]:
import org.apache.spark.sql.functions._
val getProbability = udf((prediction: org.apache.spark.ml.linalg.Vector) => prediction(1))
val sameModel = PipelineModel.load("/home/jovyan/models/spark-ml-model")
inputDF.writeStream.foreachBatch { (batchDF: DataFrame, batchId: Long) =>
    //batchDF.show()
    //batchDF.select($"tweet").show
    sameModel.transform(batchDF).groupBy($"prediction").count.show
    //sameModel.transform(batchDF).groupBy(window($"prediction", "1 minutes", "10 seconds")).count().show
    //sameModel.transform(batchDF).select($"tweet", $"probability").show
    //sameModel.transform(batchDF).select($"tweet",getProbability($"probability")).show
    
    //batchDF.select(getProbability($"probability")).show
}.trigger(Trigger.ProcessingTime("10 seconds")).outputMode("append").start()

Name: org.apache.spark.sql.AnalysisException
Message: Complete output mode not supported when there are no streaming aggregations on streaming DataFrames/Datasets;;
FileSource[/home/jovyan/data/events-stream]
StackTrace: FileSource[/home/jovyan/data/events-stream]
  at org.apache.spark.sql.catalyst.analysis.UnsupportedOperationChecker$.org$apache$spark$sql$catalyst$analysis$UnsupportedOperationChecker$$throwError(UnsupportedOperationChecker.scala:389)
  at org.apache.spark.sql.catalyst.analysis.UnsupportedOperationChecker$.checkForStreaming(UnsupportedOperationChecker.scala:117)
  at org.apache.spark.sql.streaming.StreamingQueryManager.createQuery(StreamingQueryManager.scala:250)
  at org.apache.spark.sql.streaming.StreamingQueryManager.startQuery(StreamingQueryManager.scala:316)
  at org.apache.spark.sql.streaming.DataStreamWriter.start(DataStreamWriter.scala:297)